In [18]:
import pandas as pd
from glob import glob
import re
import os

In [19]:
csv_files = glob('../csv/EXA_FIRST_マイジャグラーV_*.csv')
len(csv_files), csv_files[0]

(148, '../csv\\EXA_FIRST_マイジャグラーV_2024-10-27.csv')

In [20]:
def extract_information(csv_file):
    pattern = r"(.+)_(.+)_(\d{4}-\d{2}-\d{2})\.csv"

    file_name = os.path.basename(csv_file)
    # print(file_name)
    match = re.match(pattern, file_name)

    if match:
        hall_name, machine_name, date_str = match.groups()
        # print(f"ホール名: {hall_name}")
        # print(f"機種名: {machine_name}")
        # print(f"日付: {date}")
        return hall_name, machine_name, date_str
    else:
        print("ファイル名の形式が一致しません。")


In [21]:
dataframes = []
for csv_file in csv_files:
    hall_name, machine_name, date_str = extract_information(csv_file)
    df = pd.read_csv(csv_file, encoding="utf-8-sig")
    df["date"] = date_str
    df["hall_name"] = hall_name
    df["machine_name"] = machine_name
    dataframes.append(df)

# すべてのデータを結合したい場合
if dataframes:
    df = pd.concat(dataframes, ignore_index=True)
df.shape, df['date'].dtype


((11988, 11), dtype('O'))

In [22]:
df.to_csv('../analysis_data.csv', index=False)
df.head()

,台番号,G数,差枚,BB,RB,合成確率,BB確率,RB確率,date,hall_name,machine_name
0,1001,"5,273","-1,512",16,9.0,1/210.9,1/329.6,1/585.9,2024-10-27,EXA_FIRST,マイジャグラーV
1,1002,"6,086",-168,21,19.0,1/152.2,1/289.8,1/320.3,2024-10-27,EXA_FIRST,マイジャグラーV
2,1003,"3,371","-1,406",6,10.0,1/210.7,1/561.8,1/337.1,2024-10-27,EXA_FIRST,マイジャグラーV
3,1004,"6,835",+856,27,24.0,1/134.0,1/253.1,1/284.8,2024-10-27,EXA_FIRST,マイジャグラーV
4,1005,"3,422",-291,12,8.0,1/171.1,1/285.2,1/427.8,2024-10-27,EXA_FIRST,マイジャグラーV


In [43]:
df["G数"] = (
    df["G数"]
    .astype(str)  # 文字列型に変換
    .str.replace(",", "", regex=True)  # カンマを削除
    .str.extract(r"(\d+)")  # 数字部分のみ抽出
    .dropna()  # NaNを削除
    .astype(int)  # 整数型に変換
)
df["差枚"] = (
    df["差枚"]
    .astype(str)  # 文字列型に変換
    .str.replace(",", "", regex=True)  # カンマを削除
    .str.extract(r"(\d+)")  # 数字部分のみ抽出
    .dropna()  # NaNを削除
    .astype(int)  # 整数型に変換
)
df["合成確率"] = (
    df["合成確率"]
    .astype(str)  # 文字列型に変換
    .str.replace("1/", "", regex=True)  # カンマを削除
    .str.extract(r"(\d+)")  # 数字部分のみ抽出
    .dropna()  # NaNを削除
    .astype(int)  # 整数型に変換
)
df["BB確率"] = (
    df["BB確率"]
    .astype(str)  # 文字列型に変換
    .str.replace("1/", "", regex=True)  # カンマを削除
    .str.extract(r"(\d+)")  # 数字部分のみ抽出
    .dropna()  # NaNを削除
    .astype(int)  # 整数型に変換
)
df["RB確率"] = (
    df["RB確率"]
    .astype(str)  # 文字列型に変換
    .str.replace("1/", "", regex=True)  # カンマを削除
    .str.extract(r"(\d+)")  # 数字部分のみ抽出
    .dropna()  # NaNを削除
    .astype(int)  # 整数型に変換
)

df["date"] = pd.to_datetime(df["date"], errors="coerce")
df["month"] = df["date"].dt.to_period("M")
df['weekday'] = df['date'].dt.weekday
df['day'] = df['date'].dt.day

In [44]:
df.head()

,台番号,G数,差枚,BB,RB,合成確率,BB確率,RB確率,date,hall_name,machine_name,month,weekday,day
0,1001,5273,1512,16,9.0,210,329,585.0,2024-10-27,EXA_FIRST,マイジャグラーV,2024-10,6,27
1,1002,6086,168,21,19.0,152,289,320.0,2024-10-27,EXA_FIRST,マイジャグラーV,2024-10,6,27
2,1003,3371,1406,6,10.0,210,561,337.0,2024-10-27,EXA_FIRST,マイジャグラーV,2024-10,6,27
3,1004,6835,856,27,24.0,134,253,284.0,2024-10-27,EXA_FIRST,マイジャグラーV,2024-10,6,27
4,1005,3422,291,12,8.0,171,285,427.0,2024-10-27,EXA_FIRST,マイジャグラーV,2024-10,6,27


In [45]:
march_df = df[(df['date'] >= pd.to_datetime("2025-03-01")) & (df["date"] <= pd.to_datetime("2025-03-31"))]
march_summary = march_df.groupby("台番号")["差枚"].sum().reset_index().sort_values("差枚", ascending=False)
print(march_summary.to_string())

     台番号     差枚
19  1020  32578
68  1087  28882
44  1045  28730
10  1011  28566
53  1072  27916
58  1077  27861
48  1049  27240
62  1081  27055
80  1099  25238
2   1003  25035
6   1007  24478
75  1094  24386
69  1088  24276
61  1080  24221
31  1032  24028
64  1083  23842
55  1074  23769
33  1034  23464
56  1075  23349
26  1027  23335
8   1009  23160
47  1048  23128
39  1040  22747
18  1019  22596
60  1079  22587
59  1078  22537
43  1044  22336
35  1036  22261
20  1021  22062
29  1030  22042
63  1082  21899
71  1090  21758
54  1073  21626
77  1096  21326
51  1070  21308
79  1098  21186
1   1002  21171
21  1022  20972
5   1006  20957
17  1018  20714
74  1093  20476
15  1016  20284
13  1014  20243
25  1026  20231
9   1010  20197
73  1092  19994
14  1015  19985
24  1025  19904
72  1091  19563
32  1033  19541
66  1085  19460
42  1043  19440
57  1076  19410
38  1039  19243
49  1050  19048
3   1004  19012
30  1031  18805
65  1084  18688
36  1037  18658
67  1086  18557
7   1008  18538
76  1095

In [46]:
# データを表示（長い場合は全表示）
# pd.set_option('display.max_rows', None)  # 全行表示
# pd.set_option('display.max_columns', None)  # 全列表示

df_pivot_table = df.pivot_table(index="台番号", columns="month", values="差枚", aggfunc="sum")

for c in df_pivot_table.columns[1:-1]:
    print(c)
    display(df_pivot_table.sort_values(c, ascending=False).head(5))


2024-11


month,2024-10,2024-11,2024-12,2025-01,2025-02,2025-03
台番号,,,,,,
1035,5476,38043,31500,23823,24303,17715
1094,2135,37082,26843,23889,27108,24386
1080,2758,35851,27072,34999,20927,24221
1078,3320,34056,16059,24998,17776,22537
1099,4471,32825,23858,28068,28787,25238


2024-12


month,2024-10,2024-11,2024-12,2025-01,2025-02,2025-03
台番号,,,,,,
1077,5071,24690,38864,29426,23566,27861
1072,6061,31940,38314,28803,21685,27916
1075,3192,24207,35230,27802,23828,23349
1011,5930,26108,32817,30802,23446,28566
1050,2891,27227,32759,31308,34569,19048


2025-01


month,2024-10,2024-11,2024-12,2025-01,2025-02,2025-03
台番号,,,,,,
1080,2758,35851,27072,34999,20927,24221
1020,3682,27060,23447,34676,22150,32578
1047,4786,25237,19996,34467,19817,15317
1088,5482,30789,29345,34357,24007,24276
1032,8453,29858,19738,34272,21793,24028


2025-02


month,2024-10,2024-11,2024-12,2025-01,2025-02,2025-03
台番号,,,,,,
1050,2891,27227,32759,31308,34569,19048
1090,5875,18645,25221,23455,29292,21758
1076,4706,25095,27935,27902,29017,19410
1099,4471,32825,23858,28068,28787,25238
1073,4741,29769,28693,26063,27996,21626


In [47]:
# データを表示（長い場合は全表示）
# pd.set_option('display.max_rows', None)  # 全行表示
# pd.set_option('display.max_columns', None)  # 全列表示

df_pivot_table = df.pivot_table(index="台番号", columns="month", values="RB確率", aggfunc="mean", margins=True, margins_name="合計")

for c in df_pivot_table.columns[1:-1]:
    print(c)
    display(df_pivot_table.sort_values(c, ascending=True).head(5))


2024-11


month,2024-10,2024-11,2024-12,2025-01,2025-02,2025-03,合計
台番号,,,,,,,
1018,444.2,330.500000,452.066667,425.419355,380.615385,491.64,415.462585
1071,377.2,333.133333,331.933333,471.935484,525.423077,412.40,411.149660
1013,202.8,342.833333,432.700000,465.354839,381.653846,421.08,402.421769
1088,420.2,347.200000,396.300000,450.419355,559.807692,407.32,429.299320
1094,371.4,348.500000,415.166667,448.419355,430.807692,384.48,404.632653


2024-12


month,2024-10,2024-11,2024-12,2025-01,2025-02,2025-03,合計
台番号,,,,,,,
1071,377.2,333.133333,331.933333,471.935484,525.423077,412.40,411.149660
1002,346.4,476.066667,345.000000,420.935484,427.692308,448.68,420.068027
1039,320.8,410.133333,351.800000,399.967742,438.346154,380.40,392.979592
1079,402.0,435.000000,358.300000,409.483871,491.000000,401.20,417.000000
1078,221.0,431.500000,360.133333,425.967742,374.500000,462.40,403.782313


2025-01


month,2024-10,2024-11,2024-12,2025-01,2025-02,2025-03,合計
台番号,,,,,,,
1004,331.2,486.900000,381.900000,317.000000,408.000000,463.28,406.374150
1047,541.0,363.300000,409.233333,343.580645,404.153846,387.44,385.891156
1081,863.0,468.300000,415.566667,345.451613,468.769231,427.68,438.231293
1015,383.0,377.066667,379.666667,346.516129,428.346154,428.28,389.136054
1028,294.8,442.900000,405.600000,348.838710,489.961538,383.68,408.666667


2025-02


month,2024-10,2024-11,2024-12,2025-01,2025-02,2025-03,合計
台番号,,,,,,,
1029,397.8,382.066667,414.300000,448.193548,306.769231,416.20,395.612245
1031,353.2,419.433333,388.600000,439.225806,326.076923,354.56,387.517007
1096,367.8,420.100000,417.300000,432.645161,345.769231,422.64,407.680272
1019,388.4,464.033333,487.766667,440.290323,351.461538,382.24,427.476190
1003,306.8,407.266667,515.000000,499.064516,361.538462,392.88,434.659864


2025-03


month,2024-10,2024-11,2024-12,2025-01,2025-02,2025-03,合計
台番号,,,,,,,
1095,257.8,474.666667,377.933333,408.935484,383.538462,325.04,392.122449
1086,439.0,395.433333,409.066667,359.225806,480.653846,344.96,398.551020
1031,353.2,419.433333,388.600000,439.225806,326.076923,354.56,387.517007
1023,425.2,431.266667,461.600000,386.741935,375.576923,354.96,405.034014
1017,319.4,370.000000,490.500000,439.677419,382.961538,357.24,407.687075


In [52]:
# データを表示（長い場合は全表示）
# pd.set_option('display.max_rows', None)  # 全行表示
# pd.set_option('display.max_columns', None)  # 全列表示

df_pivot_table = df.pivot_table(index="台番号", columns="month", values="合成確率", aggfunc="mean", margins=True, margins_name="合計")

for c in df_pivot_table.columns[1:-1]:
    print(c)
    display(df_pivot_table.sort_values(c, ascending=True).head(5))


2024-11


month,2024-10,2024-11,2024-12,2025-01,2025-02,2025-03,合計
台番号,,,,,,,
1084,157.0,162.366667,191.800000,175.935484,168.407407,187.52,176.344595
1093,163.6,162.900000,180.133333,168.387097,167.629630,189.96,173.000000
1009,136.2,163.966667,169.866667,198.516129,170.370370,166.72,173.094595
1021,148.8,164.200000,196.466667,177.354839,175.111111,158.40,173.986486
1094,160.2,164.300000,182.133333,190.612903,184.740741,167.48,177.554054


2024-12


month,2024-10,2024-11,2024-12,2025-01,2025-02,2025-03,合計
台番号,,,,,,,
1043,243.8,184.966667,155.300000,184.258065,165.666667,167.08,174.250000
1047,231.8,173.266667,155.866667,200.741935,166.333333,173.40,176.229730
1042,157.4,175.400000,162.000000,171.870968,157.814815,163.56,166.128378
1035,265.0,187.133333,162.800000,161.580645,165.740741,183.64,174.986486
1088,211.4,168.933333,163.266667,183.548387,213.259259,179.48,182.148649


2025-01


month,2024-10,2024-11,2024-12,2025-01,2025-02,2025-03,合計
台番号,,,,,,,
1076,205.4,170.366667,184.633333,158.064516,155.592593,185.84,171.783784
1075,145.4,189.933333,172.233333,160.580645,164.259259,157.96,168.608108
1096,172.4,203.633333,164.300000,160.580645,186.777778,179.68,178.466216
1080,154.0,177.766667,182.933333,160.612903,167.962963,179.40,172.905405
1035,265.0,187.133333,162.800000,161.580645,165.740741,183.64,174.986486


2025-02


month,2024-10,2024-11,2024-12,2025-01,2025-02,2025-03,合計
台番号,,,,,,,
1040,198.2,166.766667,165.266667,189.387097,153.629630,193.96,174.459459
1085,160.6,170.500000,166.900000,181.967742,153.740741,172.52,169.121622
1025,131.4,182.466667,166.000000,168.903226,154.111111,209.40,173.939189
1073,191.4,169.933333,169.233333,169.387097,155.296296,158.04,165.722973
1076,205.4,170.366667,184.633333,158.064516,155.592593,185.84,171.783784


2025-03


month,2024-10,2024-11,2024-12,2025-01,2025-02,2025-03,合計
台番号,,,,,,,
1075,145.4,189.933333,172.233333,160.580645,164.259259,157.96,168.608108
1073,191.4,169.933333,169.233333,169.387097,155.296296,158.04,165.722973
1021,148.8,164.200000,196.466667,177.354839,175.111111,158.40,173.986486
1078,170.6,199.300000,173.466667,178.193548,162.222222,159.52,175.189189
1003,171.2,200.633333,166.433333,171.354839,160.259259,161.88,172.662162


In [53]:
# '台番号', 'G数', '差枚', 'BB', 'RB', '合成確率', 'BB確率', 'RB確率', 'date', 'hall_name', 'machine_name', 'month'
df_game = df.pivot_table(index="台番号", columns="month", values=["G数"], aggfunc="sum")
df_total = df.pivot_table(index="台番号", columns="month", values=["差枚"], aggfunc="sum")

for g, t in zip(df_game.columns[1:], df_total.columns[1:]):
    print(g)
    display(df_game.sort_values(g, ascending=False).head(5))
    print(t)
    display(df_total.sort_values(t, ascending=False).head(5))


('G数', Period('2024-11', 'M'))


G数                                        
month 2024-10 2024-11 2024-12 2025-01 2025-02 2025-03
台番号                                                  
1085    26439  169588  148822  148150  145677  116117
1093    18491  159389  141000  168057  127136  124172
1050    22710  158913  135895  149020  121970  119926
1092    29478  156256  141501  154885  125233  128985
1033    19274  155730  144737  158781  136907  118538

('差枚', Period('2024-11', 'M'))


差枚                                        
month 2024-10 2024-11 2024-12 2025-01 2025-02 2025-03
台番号                                                  
1035     5476   38043   31500   23823   24303   17715
1094     2135   37082   26843   23889   27108   24386
1080     2758   35851   27072   34999   20927   24221
1078     3320   34056   16059   24998   17776   22537
1099     4471   32825   23858   28068   28787   25238

('G数', Period('2024-12', 'M'))


...

差枚                                        
month 2024-10 2024-11 2024-12 2025-01 2025-02 2025-03
台番号                                                  
1050     2891   27227   32759   31308   34569   19048
1090     5875   18645   25221   23455   29292   21758
1076     4706   25095   27935   27902   29017   19410
1099     4471   32825   23858   28068   28787   25238
1073     4741   29769   28693   26063   27996   21626

('G数', Period('2025-03', 'M'))


G数                                        
month 2024-10 2024-11 2024-12 2025-01 2025-02 2025-03
台番号                                                  
1072    27936  152818  144962  142066  130565  138789
1073    23218  130055  140744  149485  138330  135012
1042    25168  140221  144132  149629  135711  131342
1041    23379  153614  152103  150012  139421  129989
1092    29478  156256  141501  154885  125233  128985

('差枚', Period('2025-03', 'M'))


差枚                                        
month 2024-10 2024-11 2024-12 2025-01 2025-02 2025-03
台番号                                                  
1020     3682   27060   23447   34676   22150   32578
1087     5224   25327   22397   28062   17588   28882
1045     3745   25429   30146   24817   14145   28730
1011     5930   26108   32817   30802   23446   28566
1072     6061   31940   38314   28803   21685   27916

In [57]:
# '台番号', 'G数', '差枚', 'BB', 'RB', '合成確率', 'BB確率', 'RB確率', 'date', 'hall_name', 'machine_name', 'month'
df_game = df.pivot_table(index="台番号", columns="weekday", values=["G数"], aggfunc="sum")
df_total = df.pivot_table(index="台番号", columns="weekday", values=["差枚"], aggfunc="sum")

for g, t in zip(df_game.columns[1:], df_total.columns[1:]):
    print(g)
    display(df_game.sort_values(g, ascending=False).head(5))
    print(t)
    display(df_total.sort_values(t, ascending=False).head(5))


('G数', 1)


G数                                              
weekday       0       1       2      3      4       5       6
台番号                                                          
1027     100316  120210  115353  92111  87357  104697  101658
1032     116143  119315  121314  73407  73890  110190  104435
1020     111569  115903   99168  84167  86455  105781  117712
1085     101601  115776  129147  79912  99673  110853  117831
1072      99961  112264  106963  85128  96851  114278  121691

('差枚', 1)


差枚                                          
weekday      0      1      2      3      4      5      6
台番号                                                     
1019     16485  29673  20633  15857  19303  18606  12824
1027     14999  26875  17974  16764  15441  18470  20028
1030     16948  26842  10091  16366  15294  21084  23549
1077     18459  26537  20843  17135  21351  22687  22466
1079     12278  26293  21316  17734  14149  14324  15569

('G数', 2)


...

差枚                                          
weekday      0      1      2      3      4      5      6
台番号                                                     
1080     20102  19643  20247  18154  17939  30137  19606
1034     14479  14502  14672  14970  14079  26469  21131
1072     20921  23580  20678  18916  21373  26466  22785
1020     25274  22023  15388  18149  17438  24625  20696
1090     12170  15624  15815  21298  15127  24181  20031

('G数', 6)


G数                                               
weekday       0       1       2      3       4       5       6
台番号                                                           
1077     114201  100703   98031  80334  111783  107957  131507
1092      93373  106343   87815  98567  107787  118924  123529
1072      99961  112264  106963  85128   96851  114278  121691
1081      90208   86973  112564  75546   89764  102759  121547
1034      76071   82438   78127  72390   92770  104386  120844

('差枚', 6)


差枚                                          
weekday      0      1      2      3      4      5      6
台番号                                                     
1094     23203  17847  20313  11347  20043  22597  26093
1011     17556  21163  30854  16404  22377  13286  26029
1017     13108  16746  16831  14560  13044  19719  25017
1098     13496  15444  17878  13782  23728  20802  24207
1036     17630  19927  13670  13297  23133  19128  23691

In [60]:
# '台番号', 'G数', '差枚', 'BB', 'RB', '合成確率', 'BB確率', 'RB確率', 'date', 'hall_name', 'machine_name', 'month'
pd.set_option('display.max_rows', None)  # 全行表示
pd.set_option('display.max_columns', None)  # 全列表示

df_game = df.pivot_table(index="台番号", columns="day", values=["G数"], aggfunc="sum")
df_total = df.pivot_table(index="台番号", columns="day", values=["差枚"], aggfunc="sum")

for g, t in zip(df_game.columns[1:], df_total.columns[1:]):
    if g[1] == 29:
        print(g)
        display(df_game.sort_values(g, ascending=False).head(5))
        print(t)
        display(df_total.sort_values(t, ascending=False).head(5))


('G数', 29)


G数                                                                 \
day      1      2      3      4      5      6      7      8      9      10   
台番号                                                                          
1077  21776  23307  25459  22999  33487  19918  19293  25797  18596  26528   
1099  15736  36418  19131  21632  30304  20016  23159  16377  19445  21558   
1079  22352  27370  15858  20537  26360  19745  18896  20059  21693  16453   
1019  21416  19532  15477  22123  25516  18060  19930  18723  21442  10289   
1047  21184  22558  20886  13452  30876  16456  20722  22562  18802  19885   

                                                                            \
day      11     12     13     14     15     16     17     18     19     20   
台番号                                                                          
1077  37576  24762  17382  16830  33430  19044  35760  15943  13390  21834   
1099  32487  24209  24709  20291  30475  24517  25858  19441  26285  25820   
1079  31859  24213  20477  21160  29173  19216  24261  22745  16941  16629   
1019  28140  19171  20940  24674  32540  12075  17275  10561  28056   9135   
1047  28693  28265  23897  15546  34988  25709  21774  24123  17009  21927   

                                                                            \
day      21     22     23     24     25     26     27     28     29     30   
台番号                                                                          
1077  31605  29326  25145  26859  34183  26948  30057   6153  26505  16021   
1099  22251  24945  24155  17750  23905  34889  15107  18993  24767  18200   
1079  31968  13621  21066  12269  33052  33721  21763  11445  24559   5454   
1019  26812  21473  25549   7004  18123  39175  15869   9019  23719  11612   
1047  24792  19727  21590  26695  27610  35130  19419  14998  23682  10848   

             
day      31  
台番号          
1077  18603  
1099  16418  
1079  17453  
1019  11460  
1047   6679

('差枚', 29)


差枚                                                                    \
day     1     2     3     4     5     6     7     8     9     10    11    12   
台番号                                                                            
1019  2830  5564  5665  5324  3897  6514  3026  3323  2503  4202  9203  4285   
1077  5962  3585  8053  5908  3344  3623  4541  4157  3339  2994  7468  7386   
1032  2421  4092  5957  5505  5970  6353  5301  3348  4070  5487  4693  4657   
1001  6541  3852  2495  4762  6973  1826  2858  3791  5792  4692  3977  6124   
1014  5220  7721  1370  2380  6400  5338  2835  3000  3679  4518  5291  1550   

                                                                              \
day     13    14    15    16    17    18    19    20    21    22    23    24   
台番号                                                                            
1019  2354  4000  4482  2415  2426  4944  3936  2894  3121  4100  3535  3162   
1077  3574  3505  7039  6122  6047  3305  3482  6036  9886  3791  2279  3888   
1032  3201  4538  3832  5738  3491  3668  6571  1892  4439  7489  4849  2750   
1001  4824  3035  5353  3329  3038  3320  2980  4786  3112  3010  1623  3456   
1014  5082  5502  4929  6004  2418  4727  1850  1794  3409  6588  2291  3457   

                                                 
day     25     26    27    28    29    30    31  
台番号                                              
1019  2386  12865  4662  2496  7847  3032  2388  
1077  7741   4901  4194  3466  6629  1059  2174  
1032  2120   7126  3405  5009  5926  1991  2253  
1001  5971   4997  3394  2873  5827  3006  4859  
1014  3350   2847  4285  2504  5503  3518  2356

In [91]:
df_target = df[(df['G数'] >= 3000) & (df['month'] == '2025-01')]
df_target = df[(df['G数'] >= 3000)]

df_total_rate = df_target.pivot_table(index="台番号", columns="day", values=["合成確率"], aggfunc="mean")
df_rb_rate = df_target.pivot_table(index="台番号", columns="day", values=["RB確率"], aggfunc="mean")

for tr, rr in zip(df_total_rate.columns[1:], df_rb_rate.columns[1:]):
    if tr[1] == 29:
        print(tr)
        display(df_total_rate.sort_values(tr, ascending=True).head(5))
        print(rr)
        display(df_rb_rate.sort_values(rr, ascending=True).head(5))

('合成確率', 29)


合成確率                                                      \
day           1           2       3           4       5           6    
台番号                                                                    
1032  146.666667  153.250000  139.50  193.000000  140.80  127.000000   
1023  140.000000  139.000000  147.25  123.000000  232.00  153.000000   
1019  132.000000  136.666667  155.00  159.400000  191.75  114.000000   
1022  137.000000  152.750000  181.00  129.666667  155.60  143.000000   
1076  136.333333  167.200000  146.00  145.750000  163.20  177.666667   

                                                                         \
day           7           8           9           10     11          12   
台番号                                                                       
1032  138.000000  173.000000  179.000000  153.500000  146.8  148.000000   
1023  143.500000  120.000000  159.000000  149.000000  161.2  151.000000   
1019  155.666667  147.333333  140.666667  156.000000  167.6  130.500000   
1022  163.500000  154.000000  156.000000  166.000000  150.4  127.000000   
1076  139.500000  129.000000  147.333333  164.333333  150.8  125.666667   

                                                                    \
day           13          14     15          16     17          18   
台番号                                                                  
1032  249.000000  162.666667  174.0  168.000000  154.0  161.333333   
1023  163.000000  152.000000  151.0  138.000000  145.5  110.000000   
1019  144.666667  132.000000  141.8  171.000000  158.5  135.000000   
1022  127.250000  141.000000  195.2  172.666667  175.4  130.666667   
1076  169.000000  153.666667  155.4  167.000000  153.4  149.000000   

                                                                            \
day           19     20          21     22      23          24          25   
台番号                                                                          
1032  153.666667  157.0  157.600000  172.0  158.00  152.250000  165.500000   
1023  141.000000    NaN  131.600000  138.0  160.00  147.000000  144.400000   
1019  162.400000  222.0  152.250000  147.0  151.75         NaN  168.666667   
1022  137.500000  149.5  135.333333  141.0  206.00  139.000000  139.800000   
1076  148.666667  140.8  179.000000  134.5  167.25  155.666667  154.250000   

                                                                
day           26      27     28     29          30          31  
台番号                                                             
1032  148.400000  172.00  143.0  112.0  147.000000  164.333333  
1023  142.666667  153.00  149.0  121.0  123.000000  135.000000  
1019  124.200000  137.00  169.0  122.0  175.000000  164.000000  
1022  163.200000  154.00  143.0  122.0  134.666667  162.000000  
1076  121.400000  166.75  168.0  122.0  141.000000  149.666667

('RB確率', 29)


RB確率                                                         \
day           1      2           3           4           5           6    
台番号                                                                       
1030  382.333333  423.5  350.750000  312.333333  433.000000  299.000000   
1013  287.666667  468.0  355.750000  440.500000  290.333333  564.666667   
1009  304.666667  319.5  454.200000  295.500000  632.250000  433.500000   
1006  333.666667  422.5  318.333333         NaN  470.750000  417.666667   
1028  337.000000  435.0  470.000000  381.666667  524.400000  322.500000   

                                                                          \
day           7           8           9           10      11          12   
台番号                                                                        
1030  331.800000  428.500000  374.000000  363.000000  324.75  337.000000   
1013  465.000000  335.666667  348.000000  317.000000  371.50  279.500000   
1009  326.333333  380.333333  425.000000  347.333333  331.60  448.500000   
1006  410.333333  280.333333  375.250000  307.000000  438.20  343.666667   
1028  314.000000  398.000000  244.333333  330.000000  315.20  270.500000   

                                                                           \
day      13          14          15          16     17     18          19   
台番号                                                                         
1030  349.0  324.333333  297.000000  424.500000  275.0  340.0  281.500000   
1013  365.0  256.000000  414.333333  395.333333  363.8    NaN  362.333333   
1009    NaN  399.400000  427.200000  481.750000  401.5  319.0  327.000000   
1006  374.0  329.000000  350.800000  386.000000  523.0  290.0  396.333333   
1028  373.0  341.250000  382.600000  384.000000  336.0  508.0  385.000000   

                                                                              \
day           20          21          22          23          24          25   
台番号                                                                            
1030  379.000000  398.500000  386.000000  495.000000  289.333333  290.750000   
1013  375.500000  397.800000  409.000000  306.000000  315.500000  323.600000   
1009  333.000000  520.000000  535.666667  351.500000  386.666667  404.400000   
1006  351.000000  583.333333  282.000000  610.333333  423.333333  442.600000   
1028  552.333333  378.000000  389.000000  330.333333  542.333333  273.666667   

                                                               
day      26          27          28     29     30          31  
台番号                                                            
1030  336.0  363.250000  276.333333  234.0  333.0  405.666667  
1013  382.0  365.333333  279.000000  241.5  257.0  330.666667  
1009  315.2  350.333333  307.000000  242.5  318.0  353.000000  
1006  355.4  326.000000  285.000000  249.0  317.5  560.500000  
1028  243.4  322.250000         NaN  259.0    NaN  419.000000